# 参数解析器
**这里我们使用现成的标准库argparse来进行参数解析任务**

In [6]:
import argparse,sys


# 创建 ArgumentParser 对象，描述用于生成 --help 的说明文字
parser = argparse.ArgumentParser(
    description='图像处理工具：支持裁剪、缩放、调整亮度、指定输出格式等功能'
)

# ===== 1. 位置参数（必填） =====
parser.add_argument(
    'input_file',
    type=str,
    help='输入图片路径（必填）'
)

# ===== 2. 基本可选参数 =====
parser.add_argument(
    '-o', '--output',
    type=str,
    default='output.jpg',
    help='输出文件路径（默认：output.jpg）'
)

# ===== 3. 类型控制与默认值 =====
parser.add_argument(
    '--scale',
    type=float,
    default=1.0,
    help='缩放比例，例如 0.5 表示缩小一半'
)

# ===== 4. 布尔值（action） =====
parser.add_argument(
    '-v', '--verbose',
    action='store_true',
    help='是否打印详细过程信息'
)

# ===== 5. 选项限制（choices） =====
parser.add_argument(
    '--format',
    choices=['jpg', 'png', 'bmp'],
    default='jpg',
    help='输出图片格式（可选：jpg, png, bmp）'
)

# ===== 6. 多值输入（nargs） =====
parser.add_argument(
    '--crop',
    type=int,
    nargs=4,
    metavar=('X', 'Y', 'W', 'H'),
    help='裁剪区域，格式为 X Y W H'
)

# ===== 7. 多次追加（append） =====
parser.add_argument(
    '--effect',
    action='append',
    help='多次应用图像效果（如 blur, sharpen, grayscale）'
)

# ===== 8. 参数别名（dest） =====
parser.add_argument(
    '--lightness',
    dest='brightness',
    type=int,
    default=0,
    help='亮度调整（-100 到 100）'
)

# ===== 9. 可选位置参数（nargs="?"+default） =====
parser.add_argument(
    'mode',
    nargs='?',
    choices=['fast', 'accurate'],
    default='fast',
    help='处理模式（默认：fast）'
)

# ===== 10. 解析参数 =====
args, unknown = parser.parse_known_args([
    'input.jpg',
    '--format', 'png',
    '-v',
    '--effect', 'blur',
    '--effect', 'sharpen',
    '--crop', '10', '10', '100', '100',
    '--lightness', '20',
    'accurate'
]
)

if unknown:
    print("警告：未识别参数:", unknown)


# ===== 11. 使用参数（模拟处理） =====
if args.verbose:
    print(f'开始处理: {args.input_file}')
    print(f'输出文件: {args.output}')
    print(f'缩放比例: {args.scale}')
    print(f'输出格式: {args.format}')
    print(f'裁剪参数: {args.crop}')
    print(f'应用特效: {args.effect}')
    print(f'亮度调整: {args.brightness}')
    print(f'处理模式: {args.mode}')

# 模拟输出
print(f"[✓] 已处理图片并输出至：{args.output}")




警告：未识别参数: ['accurate']
开始处理: input.jpg
输出文件: output.jpg
缩放比例: 1.0
输出格式: png
裁剪参数: [10, 10, 100, 100]
应用特效: ['blur', 'sharpen']
亮度调整: 20
处理模式: fast
[✓] 已处理图片并输出至：output.jpg


# 了解YOLO模型的输入输出格式


YOLO输入输出格式特点比较固定，主要是：


## YOLO 输入格式


* **格式：** 通常是三通道的RGB图像，`大小固定`（如YOLOv3常用的输入是 416×416 或 608×608），故，图像一般需要预处理：

  1. 缩放到固定大小
  2. 归一化（像素值通常归一化到0~~1或者-1~~1）
  3. 转为模型接受的形状`shape`格式（如`[batch_size, channels, height, width]`）

      | 参数                    | 说明                              | 具体举例             |
      | --------------------- | ------------------------------- | ---------------- |
      | **batch_size**（批量大小） | 一次送入模型的图片数量。YOLO支持批量输入，提高计算效率。  | `1` 表示只输入一张图     |
      | **channels**（通道数）     | RGB,就是咱们说的三大基本色 | `3` 表示红绿蓝三通道     |
      | **height**（高度）        |图片尺度                 | `416` 表示图像高416像素 |
      | **width**（宽度）         | 图片尺度                  | `416` 表示图像宽416像素 |


* **输入一般是张量（Tensor）形式**，具体格式视深度学习框架而定，Mindspore的是
      
    | MindSpore张量特性 | 说明                    |
    | ------------- | --------------------- |
    | 多维数组          | 表示数据的多维矩阵             |
    | 支持多种dtype     | float32、int32等        |
    | 有形状信息         | 通过 `.shape` 属性查看      |
    | 兼容设备          | 可自动映射到CPU/GPU/Ascend等 |


## YOLO 输出格式

* **输出内容：** `检测框信息（边界框）`、`类别概率`和`置信度`

* **格式：** 一般是一个二维数组，形状类似 `[N, 5 + num_classes]`，其中：

  * N 是检测到的候选框数量（通常是模型预测的固定数量框）
  * 5代表框的5个参数：`(x_center, y_center, width, height, objectness_score)`

    | 参数名                   | 含义        | 说明                     |
    | --------------------- | --------- | ---------------------- |
    | **x\_center**         | 边界框中心点横坐标 | 通常相对于图像宽度的归一化坐标，范围0\~1 |
    | **y\_center**         | 边界框中心点纵坐标 | 通常相对于图像高度的归一化坐标，范围0\~1 |
    | **width**             | 边界框宽度     | 相对于图像宽度的比例或像素值         |
    | **height**            | 边界框高度     | 相对于图像高度的比例或像素值         |
    | **objectness\_score** | 目标置信度     | 该框包含物体的概率，范围0\~1       |


  * `num_classes` 是类别概率分布（每个类别对应一个概率）

* **后处理：** 模型输出的框还需要经过非极大值抑制（NMS）过滤重叠框，得到最终的检测结果。

    > **NMS（非极大值抑制）的作用：**
    > 去除目标检测中多个高度重叠的框，只保留置信度最高的那个，防止同一个目标被重复检测。
    >
    > **工作流程(了解)：**
    > 1. 按置信度排序所有候选框。
    > 2. 取置信度最高的框，输出它。
    > 3. 删除与该框重叠（IoU超过阈值）的其他框。
    > 4. 重复以上步骤，直到所有框处理完。



    这样能让检测结果更准确，避免重复框。





    * 输入：一张RGB图，尺寸416×416

    * 输出（未经NMS示意）：

      ```
      [
        [0.5, 0.5, 0.2, 0.3, 0.9, 0.1, 0.8, 0.1],  # 框中心x,y,宽高,置信度, 类别概率(3类示例)
        [0.7, 0.4, 0.1, 0.2, 0.75, 0.7, 0.2, 0.1],
        ...
      ]
      ```

    * 经过NMS后输出：

      ```
      [
        {box: [x1, y1, x2, y2], score: 0.85, class_id: 1},
        {box: [x1, y1, x2, y2], score: 0.72, class_id: 0},
        ...
      ]
      ```


# 进行图像对模型的输入

In [4]:
import cv2
import numpy as np
import mindspore

def preprocess_image(image_path, size=(416, 416)):
    # 读取图片，默认是BGR格式
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"无法读取图片: {image_path}")
    
    # 调整大小
    img_resized = cv2.resize(img, size)
    
    # BGR转RGB
    img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)
    
    # 归一化到0~1 ，将0-255 映射到 0-1之间的小数
    img_norm = img_rgb.astype(np.float32) / 255.0
    
    # HWC转CHW格式 以供模型支持
    #(H,W,C)
        # H = 高度（Height）
        # W = 宽度（Width）
        # C = 通道数（Channel，通常是3，即RGB）


    img_chw = np.transpose(img_norm, (2, 0, 1))
    
    # | 原维度   | 新位置     |
    # | 0 (H) | 放到第 1 维 |    
    # | 1 (W) | 放到第 2 维 |    
    # | 2 (C) | 放到第 0 维 |    
    #把原来的 (H, W, C) 变成 (C, H, W)。
    
    #为什么这么转？
        # 因为大多数深度学习模型（如 PyTorch、MindSpore）要求图像的输入格式是：

            # (batch_size, channels, height, width)  # 即 BCHW 格式
            # (channels, height, width)  # 即 CHW 格式



            
    # 给图像数据增加一个批次维度batch，变成 (1, channels, height, width)
    img_batch = np.expand_dims(img_chw, axis=0)
    
    return img_batch #返回

def test_yolo_input_tensor(image_path):
    img_np = preprocess_image(image_path)
    tensor = mindspore.Tensor(img_np, mindspore.float32)
    
    print("输入张量的形状:", tensor.shape)  # 应该是 (1, 3, 416, 416)
    print("输入张量的数据类型:", tensor.dtype)
    print("张量示例数据 (第一个像素点RGB):", tensor[0, :, 0, 0].asnumpy())

if __name__ == "__main__":
    image_path = "test_by_yolov5s\images\images.jpg"  # 换成你本地的图片路径
    test_yolo_input_tensor(image_path)


输入张量的形状: (1, 3, 416, 416)
输入张量的数据类型: Float32
张量示例数据 (第一个像素点RGB): [0.17254902 0.17254902 0.16470589]


# 进行模型推理

### 先导入一个参数解析器

In [12]:
import argparse

def get_args():
    parser = argparse.ArgumentParser(description="YOLOv5 MindSpore 推理脚本参数")

    # 图片路径或视频帧路径
    parser.add_argument('--image_path', type=str, required=True,
                        help='输入图像文件路径')

    # 模型相关
    parser.add_argument('--weight', type=str, required=True,
                        help='模型权重文件路径 (.ckpt)')

    parser.add_argument('--model_name', type=str, default='yolov5',
                        help='模型名称，默认 yolov5')

    parser.add_argument('--img_size', type=int, default=416,
                        help='输入图像大小，默认 416')

    # 数据集和类别
    parser.add_argument('--num_classes', type=int, default=80,
                        help='类别数量，默认80')

    parser.add_argument('--dataset_name', type=str, default='coco',
                        help='数据集名称，默认 coco')

    parser.add_argument('--class_names', type=str, default='',
                        help='类别名称列表，用逗号分隔，如 person,car,bike。若为空则用数字代替')

    # 推理阈值
    parser.add_argument('--conf_thres', type=float, default=0.25,
                        help='置信度阈值，默认0.25')

    parser.add_argument('--iou_thres', type=float, default=0.45,
                        help='NMS阈值，默认0.45')

    parser.add_argument('--conf_free', action='store_true',
                        help='是否使用conf_free模式')

    parser.add_argument('--exec_nms', action='store_true',
                        help='是否执行NMS，默认执行')

    parser.add_argument('--nms_time_limit', type=float, default=0.5,
                        help='NMS时间限制，默认0.5秒')

    # 其他
    parser.add_argument('--save_result', action='store_true',
                        help='是否保存推理结果')

    parser.add_argument('--save_dir', type=str, default='./results',
                        help='保存结果的目录，默认 ./results')

    parser.add_argument('--ms_amp_level', type=str, default='O0',
                        help='MindSpore 自动混合精度等级，默认 O0')

    # 网络配置（简单示例，实际项目可根据需求扩展）
    parser.add_argument('--stride', type=int, nargs='+', default=[32],
                        help='模型步长列表，默认[32]')

    args = parser.parse_args()




# 模型推理

关于**模型推理过程**，可以拆成几个关键步骤。简单说，就是：




### 1. **模型定义（网络结构搭建）**

* 先用代码描述神经网络结构
* 框架根据定义创建模型实例，准备好各层

### 2. **加载预训练权重**

* 从训练好的参数文件（checkpoint）中读取权重
* 将权重赋值给模型中对应的层
* 权重加载成功后，模型具备推理能力

### 3. **输入数据预处理**

* 读取输入数据（图片、视频帧等）
* 图像尺寸调整（resize到网络输入尺寸，比如640x640）
* 数据归一化（如除以255转为\[0,1]范围）
* 颜色通道顺序调整（如BGR转RGB）
* 变换成模型要求的格式（如NCHW，即batch, channel, height, width）
* 转成框架支持的Tensor类型

### 4. **模型前向推理**

* 把预处理后的数据输入模型
* 运行前向计算，获得输出结果（通常是多尺度预测框、类别置信度等）

### 5. **后处理**

* 对模型输出结果解码成边界框和类别标签
* 使用非极大值抑制（NMS）去除重叠框
* 过滤置信度低的检测结果
* 转换框坐标回原图尺度
* 输出最终检测框和类别

### 6. **结果展示或保存**

* 在原图上画框和类别标签
* 保存检测结果（图片、文本、json等）
* 或者用于后续的业务逻辑

---

## 总结流程图

```
输入图片
   ↓
预处理（resize/归一化/转tensor）
   ↓
模型定义 + 加载权重
   ↓
模型前向推理（得到原始预测结果）
   ↓
后处理（框解码、NMS、过滤）
   ↓
结果可视化/输出
```



NameError: name 'create_model' is not defined

| 模块分类   | API名称               | 功能描述                       | 使用示例                                                      |
|------------|-----------------------|-------------------------------|---------------------------------------------------------------|
| 模型构建   | create_model          | 创建YOLO系列模型实例          | model = create_model('yolov5s', checkpoint_path='yolov5s.ckpt') |
| 数据预处理 | letterbox             | 图像缩放填充预处理            | img = letterbox(img, new_shape=640)                           |
|            | ImageListtoTensor     | 将图像列表转换为MindSpore张量 | tensor = ImageListtoTensor([img1, img2])                      |
|            | create_dataset        | 创建训练/验证数据集           | train_dataset = create_dataset(args.data.train_set, is_train=True) |
| 推理执行   | detect                | 执行目标检测任务              | results = detect(model, img, conf_thres=0.25)                 |
|            | segment               | 执行实例分割任务              | masks = segment(model, img)                                   |
| 结果可视化 | draw_result           | 绘制检测/分割结果可视化       | vis_img = draw_result(img, results, class_names=['person', 'car']) |
|            | show_image            | Jupyter环境实时显示结果       | show_image(result_img, title='检测结果')                      |
| 配置管理   | parse_args            | 解析YAML配置文件              | args = parse_args('configs/yolov5s.yaml')                     |
|            | set_default_infer     | 设置推理默认参数              | set_default_infer(args)                                       |
| 训练管理   | create_optimizer      | 创建优化器                   | opt = create_optimizer(model, lr=0.01, momentum=0.937)        |
|            | YOLOv5Loss            | YOLO系列损失函数             | loss_fn = YOLOv5Loss(box=0.05, cls=0.5, obj=1.0)              |
|            | create_lr_scheduler   | 创建学习率调度器              | lr_scheduler = create_lr_scheduler(lr_init=0.01, warmup_epochs=3) |
|            | auto_mixed_precision  | 自动混合精度训练              | ms.amp.auto_mixed_precision(network, amp_level="O2")          |
| 设备管理   | sync_data             | 跨设备/云脑数据同步           | sync_data('obs://dataset/', './local_data')                    |
| 日志系统   | logger                | 统一日志记录接口              | logger.info("Inference completed")                            |
|            | TensorBoardLogger     | 训练指标可视化                | logger = TensorBoardLogger(log_dir='./logs')                   |
| 工具函数   | set_seed              | 设置全局随机种子              | set_seed(42)                                                  |
| 数据增强   | apply_augmentations   | 应用数据增强流水线            | aug_img = apply_augmentations(img, mosaic=True, hsv_aug=True) |
| 模型部署   | export                | 导出MindIR格式模型            | export(model, Tensor(input), file_name="model", file_format="MINDIR") |
| 视频处理   | VideoStreamHandler    | 实时视频流处理                | handler = VideoStreamHandler(camera_index=0, frame_size=(640,480)) |
